What we're doing: Install required packages for the memory lesson in Colab.

In [ ]:
!pip install -qU langchain-groq langgraph langchain-community pysqlite3-binary

What we're doing: Load the GROQ API key from Colab userdata into the environment.

In [ ]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

What we're doing: Connect to the sample SQLite database and print available tables.

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:////content/sample_data/Chinook.db")
print(db.get_usable_table_names())

What we're doing: Define `RuntimeContext` dataclass to hold the database reference for runtime use.

In [ ]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

What we're doing: Define `execute_sql` tool to allow the agent to run read-only SQL queries via the runtime.

In [ ]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

What we're doing: Set `SYSTEM_PROMPT` rules to constrain agent SQL behavior and ensure read-only queries.

In [ ]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

What we're doing: Initialize the Groq model and create an agent with `execute_sql` and runtime context.

In [ ]:
from langchain.agents import create_agent
from langchain_groq import ChatGroq

# Initialize the Groq model
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_retries=2,
)

agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

What we're doing: Stream a repeated query scenario to observe agent behavior without memory enabled.

In [ ]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

What we're doing: Follow up query without memory to show that prior context isn't retained across invocations.

In [ ]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## Add memory

What we're doing: Import an in-memory checkpointer to enable simple agent memory persistence.

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

What we're doing: Re-create the agent with an in-memory checkpointer so it can remember prior interactions.

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

What we're doing: Run the same invoice query with memory enabled (thread_id=1) to persist context between calls.

In [ ]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

What we're doing: Ask a follow-up question using the same memory `thread_id` to check recall of details like titles.

In [ ]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

## Try your own queries
Now that there is memory, check the agents recall!

What we're doing: Placeholder for you to enter your own question and test agent memory behavior.

In [ ]:
question = "Your Question Here?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

What we're doing: Example follow-up that asks the agent to show full results using the same memory thread.

In [ ]:
question = "yes i want to see the full list"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

(End of notebook)